In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Mounted at ./MyDrive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import joblib
plt.rc('font', family='NanumBarunGothic')

# keras 관련 모듈
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.model_selection import train_test_split
from keras import backend as K
from tensorflow.keras.optimizers import Adam

import os

In [3]:
# 데이터 불러오기

path = '/content/MyDrive/MyDrive/2021-2 데이터애널리틱스/'

data = pd.read_csv(path+'DaData_Add.csv', encoding='cp949', engine='python')
data

,날짜,평균강수량,최대강수량,평균기온,최고기온,최저기온,평균습도,최저습도,평균풍속,최대풍속,건고추가격,건고추거래량,배추가격,배추거래량,대파가격,대파거래량,마늘가격,마늘거래량,무가격,무거래량,고급휘발유,보통휘발유,자동차용경유,소비자물가,농축수산물,공업제품,집세,공공서비스,개인서비스,근원물가,생활물가
0,2014-01-01,0.0,0.1,4.8,9.2,-0.3,51,11,3.0,15.5,0.00000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,2202.27,1887.37,1705.85,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
1,2014-01-02,0.0,0.0,2.6,9.1,-2.2,49,10,1.6,9.3,0.00000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,2201.07,1887.56,1706.27,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
2,2014-01-03,0.0,0.0,2.6,10.1,-3.2,54,12,1.2,9.6,0.00000,0.00,471.249011,549.64,857.382976,408.89,3530.248412,77.13,307.198836,833.98,2200.04,1888.51,1707.15,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
3,2014-01-04,1.2,31.0,1.5,6.9,-2.8,63,12,1.7,11.9,0.00000,0.00,425.448876,423.95,891.743009,322.98,2409.531445,17.65,300.931167,615.89,2199.92,1888.98,1707.61,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
4,2014-01-05,0.1,30.2,0.7,6.6,-3.7,62,9,1.6,10.1,0.00000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,2199.00,1889.11,1707.56,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2552,2020-12-27,2.1,53.2,3.2,8.1,-1.0,77,18,1.0,12.2,0.00000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,1650.22,1406.84,1207.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2553,2020-12-28,0.1,1.4,4.2,11.9,-0.4,80,22,0.9,6.9,16176.22666,2.23,427.106550,889.49,1663.589524,859.43,5011.157127,77.70,465.380562,1679.14,1651.94,1409.68,1210.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2554,2020-12-29,1.8,6.9,2.1,8.6,-2.3,81,31,1.8,20.0,19287.12446,1.17,414.354554,699.63,1651.757279,797.63,4629.185781,115.58,496.425306,1002.42,1654.39,1413.82,1214.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2555,2020-12-30,1.1,33.4,-6.8,-1.1,-9.7,56,17,4.0,27.5,15623.79171,104.21,481.071567,1034.70,1829.078794,786.60,5110.230674,80.37,510.500562,965.00,1657.64,1417.11,1218.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def zero_to_mean(data, name) :
  print('원본 데이터 수 : ', len(data))
  print('평균 대체 전 ', name, '== 0 개수 : ', len(data[data[name] == 0]))

  # 가격열 0인 경우 1일전/후 데이터의 평균값으로 대체
  zero_index = data[data[name] == 0][name].index
  num1list = [1] * len(zero_index)
  zero_index = zero_index[1:-1]
  zero_pre = [zero_index[i] - num1list[i] for i in range(len(zero_index))]
  zero_post = [zero_index[i] + num1list[i] for i in range(len(zero_index))]
  print(len(zero_pre), len(zero_post))

  zero_sum = [data.loc[zero_pre, name].tolist()[i] + data.loc[zero_post, name].tolist()[i] for i in range(len(zero_index))]

  num2list = [2] * len(zero_index)
  mean_result = [zero_sum[i] / num2list[i] for i in range(len(zero_index))]
  # print(data.loc[outlier_pre, name].tolist())
  # print(data.loc[outlier_post, name].tolist())
  # print(outlier_result)
  data.loc[zero_index, name] = mean_result
  print('평균 대체 후 ', name, '== 0 개수 : ', len(data[data[name] == 0]), '\n')
  return data

def zero_index(data, name) :
  return data[data[name] == 0][name].index.tolist()

def result(data) :
  name_list = ["건고추가격", "건고추거래량", "배추가격", "배추거래량", "대파가격", "대파거래량", "마늘가격", "마늘거래량", "무가격", "무거래량"]
  for name in name_list :
    data = zero_to_mean(data, name)

  pricename_list = ["건고추가격", "배추가격", "대파가격", "마늘가격", "무가격"]
  index_list = []

  for i in pricename_list :
    index_list.append(zero_index(data, i))

  index_list = sum(index_list, []) # 2차원 -> 1차원 변환
  index_list = set(index_list) # 중복 제거
  index_list = list(index_list) # set -> list 형태로 변환
  print(index_list)
  data.drop(index_list, inplace = True)

  for i in pricename_list :
    index_list.append(zero_index(data, i))
    print('제거 후 ', i, '== 0 개수 : ', len(data[data[i] == 0]), '\n')

  return data

In [5]:
data = result(data)

원본 데이터 수 :  2557
평균 대체 전  건고추가격 == 0 개수 :  928
926 926
평균 대체 후  건고추가격 == 0 개수 :  347 

원본 데이터 수 :  2557
평균 대체 전  건고추거래량 == 0 개수 :  928
926 926
평균 대체 후  건고추거래량 == 0 개수 :  347 

원본 데이터 수 :  2557
평균 대체 전  배추가격 == 0 개수 :  389
387 387
평균 대체 후  배추가격 == 0 개수 :  19 

원본 데이터 수 :  2557
평균 대체 전  배추거래량 == 0 개수 :  389
387 387
평균 대체 후  배추거래량 == 0 개수 :  19 

원본 데이터 수 :  2557
평균 대체 전  대파가격 == 0 개수 :  388
386 386
평균 대체 후  대파가격 == 0 개수 :  16 

원본 데이터 수 :  2557
평균 대체 전  대파거래량 == 0 개수 :  388
386 386
평균 대체 후  대파거래량 == 0 개수 :  16 

원본 데이터 수 :  2557
평균 대체 전  마늘가격 == 0 개수 :  408
406 406
평균 대체 후  마늘가격 == 0 개수 :  23 

원본 데이터 수 :  2557
평균 대체 전  마늘거래량 == 0 개수 :  408
406 406
평균 대체 후  마늘거래량 == 0 개수 :  23 

원본 데이터 수 :  2557
평균 대체 전  무가격 == 0 개수 :  388
386 386
평균 대체 후  무가격 == 0 개수 :  16 

원본 데이터 수 :  2557
평균 대체 전  무거래량 == 0 개수 :  388
386 386
평균 대체 후  무거래량 == 0 개수 :  16 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 4

In [6]:
data.to_csv(path+'data_zero_to_mean.csv')

In [7]:
df = pd.read_csv(path+'data_zero_to_mean.csv', index_col=0)
df

,날짜,평균강수량,최대강수량,평균기온,최고기온,최저기온,평균습도,최저습도,평균풍속,최대풍속,건고추가격,건고추거래량,배추가격,배추거래량,대파가격,대파거래량,마늘가격,마늘거래량,무가격,무거래량,고급휘발유,보통휘발유,자동차용경유,소비자물가,농축수산물,공업제품,집세,공공서비스,개인서비스,근원물가,생활물가
21,2014-01-22,0.0,0.0,-2.7,3.4,-8.0,60,11,2.0,15.7,2925.000000,2.50,330.544001,401.230,1002.515486,277.030,2355.274786,27.11,249.686396,630.09,2195.18,1884.91,1703.67,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
22,2014-01-23,0.0,0.1,-1.0,7.1,-7.5,62,12,1.3,10.8,5850.000000,5.00,315.643399,492.540,989.747710,313.130,2305.000000,14.34,251.183856,622.96,2195.24,1884.54,1703.47,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
23,2014-01-24,0.0,0.0,4.0,10.7,-3.5,66,23,1.9,11.3,2925.000000,2.50,327.110193,487.950,955.507856,342.420,2396.097276,34.29,260.191008,586.78,2195.73,1884.02,1702.88,1.1,-5.6,1.7,2.4,0.7,1.2,1.7,0.6
44,2014-02-14,0.2,5.4,1.9,6.0,-1.6,63,26,2.3,11.2,2985.220588,2.55,267.263201,371.590,885.361794,306.670,2178.802407,19.61,251.805551,801.20,2192.90,1879.89,1697.95,1.0,-5.4,1.7,2.5,0.7,0.9,1.7,0.4
45,2014-02-15,0.0,1.0,1.7,8.9,-4.1,62,15,1.6,11.6,5970.441176,5.10,264.532134,260.710,849.281352,264.270,2030.019560,12.27,208.875774,655.40,2194.28,1879.81,1697.69,1.0,-5.4,1.7,2.5,0.7,0.9,1.7,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2551,2020-12-26,0.0,0.0,1.2,7.9,-4.4,68,23,1.2,9.5,24170.040820,0.39,450.347876,490.040,1629.113111,723.440,5258.713656,52.20,465.221469,925.88,1649.25,1405.23,1206.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2552,2020-12-27,2.1,53.2,3.2,8.1,-1.0,77,18,1.0,12.2,20173.133740,1.31,438.727213,689.765,1646.351317,791.435,5134.935392,64.95,465.301015,1302.51,1650.22,1406.84,1207.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2553,2020-12-28,0.1,1.4,4.2,11.9,-0.4,80,22,0.9,6.9,16176.226660,2.23,427.106550,889.490,1663.589524,859.430,5011.157127,77.70,465.380562,1679.14,1651.94,1409.68,1210.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2554,2020-12-29,1.8,6.9,2.1,8.6,-2.3,81,31,1.8,20.0,19287.124460,1.17,414.354554,699.630,1651.757279,797.630,4629.185781,115.58,496.425306,1002.42,1654.39,1413.82,1214.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df[df["건고추가격"] == 0]

,날짜,평균강수량,최대강수량,평균기온,최고기온,최저기온,평균습도,최저습도,평균풍속,최대풍속,건고추가격,건고추거래량,배추가격,배추거래량,대파가격,대파거래량,마늘가격,마늘거래량,무가격,무거래량,고급휘발유,보통휘발유,자동차용경유,소비자물가,농축수산물,공업제품,집세,공공서비스,개인서비스,근원물가,생활물가
